In [1]:
#Import.
import pandas as pd
import numpy as np
from ast import literal_eval
from tqdm.notebook import tqdm
from numba import njit, guvectorize, vectorize
from numpy import float64, bool, int32
from itertools import product

In [2]:
#Funciones.
def load_ib():
    name = 'ib'
    V = pd.read_csv(name+'.csv', sep=';')
    return np.array(V)

In [3]:
#Loading data.
ib = load_ib()
IB = pd.DataFrame(ib, columns=['ib'])
IB['constante'] = 1
DFF = pd.read_csv('dff.csv', sep=';')
DFF_PRED = pd.read_csv('dff_pred.csv', sep=';')
ITEM_DATA = pd.read_csv('item_data.csv', sep=';')
ITEM_DATA.loc[ITEM_DATA['product_id'] == 0, 'product_id'] = -1
ITEM_DATA['domain_code'], domain_uniques = pd.factorize(ITEM_DATA['domain_id'], sort=True)
ITEM_DATA['category_code'], category_uniques = pd.factorize(ITEM_DATA['category_id'], sort=True)

In [4]:
#Corrección temporaria.
DFF = DFF.rename(columns={'domain_id': 'domain_code', 'product_code': 'product_id'})
DFF_PRED = DFF_PRED.rename(columns={'domain_id': 'domain_code', 'product_code': 'product_id'})

In [5]:
target = np.array(IB.groupby('ib').sum().sort_values('constante', ascending=False).index)
for tipo in ['domain_code', 'product_id', 'category_code']:
    DFF[tipo] = DFF[tipo].apply(lambda x: np.array(literal_eval(x)))
    DFF_PRED[tipo] = DFF_PRED[tipo].apply(lambda x: np.array(literal_eval(x)))
DFF.loc[DFF['sd_p'] == 0, 'sd_p'] = DFF['median_p'] * 0.1
DFF_PRED.loc[DFF_PRED['sd_p'] == 0, 'sd_p'] = DFF_PRED['median_p'] * 0.1

In [6]:
A = np.array(DFF['median_p'])
B = np.array(DFF['product_id'])
D = np.array(DFF['sd_p'])

In [7]:
a = np.array(ITEM_DATA['price'])
b = np.array(ITEM_DATA['product_id'])
c = np.array(ITEM_DATA['item_id'])

In [10]:
#Reducir a los que tuvieron ventas.
a = a[np.isin(c, target, assume_unique=True)]
b = b[np.isin(c, target, assume_unique=True)]
c = c[np.isin(c, target, assume_unique=True)]

In [12]:
def super_funcion(target, a, b, c, A, B, D, g1, g2, k=10):
    return target[np.isin(target, c[(np.isin(b, B[:g1], assume_unique=True)) & (a>=A-D*g2) & (a<=A+D*g2)], assume_unique=True)][:k]

In [13]:
%%time
PRED = {}
pred = {}
K = 10

for l in product(range(1,4), range(1,4)):
    name = 'dim_spro_{}{}'.format(l[0],l[1])
    pred[name] = np.full([A.size, K], -1)
    for i in tqdm(range(A.size)):
        r = super_funcion(target, a, b, c, A[i], B[i], D[i], l[0], l[1], k=K)
        for j in range(np.min([r.size, K])):
            pred[name][i, j] = r[j]
        if i % 50000 == 0:
            PRED[name] = pd.DataFrame(pred[name], columns=['pred'+str(i) for i in range(K)])
            PRED[name].to_csv('{}_r{}.csv'.format(name, i, index=False, sep=';'))

    PRED[name] = pd.DataFrame(pred[name], columns=['pred'+str(i) for i in range(K)])
    PRED[name].to_csv('{}_r{}.csv'.format(name, i), index=False, sep=';')


Wall time: 6h 23min 54s


In [122]:
@vectorize(['boolean(float64, float64, float64)'], target='parallel', nopython=True)
def g(a, alfa, delta):
    return a >= alfa - delta and a <= alfa + delta

In [32]:
DIM_VC = pd.DataFrame(dim_vc, columns=['pred'+str(i) for i in range(10)])
DIM_VC.to_csv('dim_vc.csv', index=False, sep=';')

,0,1,2,3,4,5,6,7,8,9
0,1748830,1154325,652718,400496,1615991,2095678,473325,485055,910588,1696611
1,178545,541961,285756,1982437,1434740,1484233,605616,1468462,1879660,229369
2,182725,1748343,303917,1649564,-1,-1,-1,-1,-1,-1
3,1828725,59942,-1,-1,-1,-1,-1,-1,-1,-1
4,1361154,725371,1748830,849156,1127740,87559,817689,1839144,1647194,1847644
...,...,...,...,...,...,...,...,...,...,...
95,2032612,777316,768694,875255,1281524,856245,1661323,1120200,268262,577128
96,14022,1446348,839947,1589173,1860454,830338,338319,1990398,1984943,1226703
97,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
98,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [58]:
pred

{'dim_scat_11': array([[-1, -1, -1, ..., -1, -1, -1],
        [-1, -1, -1, ..., -1, -1, -1],
        [-1, -1, -1, ..., -1, -1, -1],
        ...,
        [-1, -1, -1, ..., -1, -1, -1],
        [-1, -1, -1, ..., -1, -1, -1],
        [-1, -1, -1, ..., -1, -1, -1]]),
 'dim_scat_12': array([[-1, -1, -1, ..., -1, -1, -1],
        [-1, -1, -1, ..., -1, -1, -1],
        [-1, -1, -1, ..., -1, -1, -1],
        ...,
        [-1, -1, -1, ..., -1, -1, -1],
        [-1, -1, -1, ..., -1, -1, -1],
        [-1, -1, -1, ..., -1, -1, -1]]),
 'dim_scat_13': array([[-1, -1, -1, ..., -1, -1, -1],
        [-1, -1, -1, ..., -1, -1, -1],
        [-1, -1, -1, ..., -1, -1, -1],
        ...,
        [-1, -1, -1, ..., -1, -1, -1],
        [-1, -1, -1, ..., -1, -1, -1],
        [-1, -1, -1, ..., -1, -1, -1]]),
 'dim_scat_21': array([[-1, -1, -1, ..., -1, -1, -1],
        [-1, -1, -1, ..., -1, -1, -1],
        [-1, -1, -1, ..., -1, -1, -1],
        ...,
        [-1, -1, -1, ..., -1, -1, -1],
        [-1, -1, -1, ...

In [57]:
PRED = {}
pred = {}
K = 10

for l in product(range(1,4), range(1,4)):
    name = 'dim_scat_{}{}'.format(l[0],l[1])
    pred[name] = np.full([A.size, 10], -1)
    
